In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
Porn=np.load("Porn_data.npy")
Non_Porn=np.load("Non_Porn_data.npy")

In [3]:
input_data=[]
for i in range(2):
    if i == 0:
        for j in range(len(Non_Porn)):
            input_data.append(Non_Porn[j])
    else :
        for j in range(len(Porn)):
            input_data.append(Porn[j])

In [4]:
input_data=np.array(input_data)

In [5]:
label=[]
for i in range(len(Non_Porn)+len(Porn)):
    if i>len(Non_Porn):
        label.append(0)
    else:
        label.append(1)

In [6]:
label=np.array(label)

In [7]:
input_data= (input_data*(1/256)-0.5)/0.5

In [12]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [13]:
input_data=input_data.transpose(1,3)

In [14]:
input_data.shape

torch.Size([136681, 3, 50, 50])

In [15]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [16]:
trainset,valset=torch.utils.data.random_split(dataset,[116681,20000])
trainset,testset=torch.utils.data.random_split(trainset,[96681,20000])

partition={'train':trainset,'val':valset,'test':testset}

In [57]:
class AlexNet_test(nn.Module):
    
    def __init__(self):
        super(AlexNet_test,self).__init__()
        self.feature=nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=2),# 64 12 12 => kernel remake
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),# 64 13 13 => kernel remake
        nn.Conv2d(64, 192, kernel_size=4, padding=2), # 192, 13, 13 => kernel remake
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2), # 192 7 7 => kernel remake
        nn.Conv2d(192, 384, kernel_size=3, padding=1), # 384 7 7 => kernel remake
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, padding=1), # 256 7 7
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2) # 256 6 6 
        )
        self.avgpool=nn.AdaptiveAvgPool2d((3,3)) # 256 6 6 (6,6)은 output size이다.
        # 평균을 구하는 것으로 반올림 정수가 나옴.
        self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256*3*3,1024),  # 9216=>4096, 256 36
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(1024,1024),
        nn.ReLU(inplace=True),
        nn.Linear(1024,2), # => 2 class
        
        )
    def forward(self, x):
        x=self.feature(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1) # ex => 2,2,2 => 2,4
        x=self.classifier(x)
        return x

In [52]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=4096,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [53]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=4096, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [54]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=4096, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [55]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [59]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=AlexNet_test()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  0%|          | 0/24 [00:00<?, ?it/s]

time : 10.26, train_loss : 0.67, train_acc : 63.47,val_acc : 81.17,val_loss : 0.45


  0%|          | 0/24 [00:00<?, ?it/s]

time : 9.64, train_loss : 0.42, train_acc : 82.52,val_acc : 82.45,val_loss : 0.41


  0%|          | 0/24 [00:00<?, ?it/s]

...

  0%|          | 0/24 [00:00<?, ?it/s]

time : 10.07, train_loss : 0.95, train_acc : 56.32,val_acc : 52.37,val_loss : 0.69


  0%|          | 0/5 [00:00<?, ?it/s]

time : 10.00, train_loss : 0.69, train_acc : 52.02,val_acc : 52.37,val_loss : 0.69


100%|██████████| 5/5 [00:00<00:00,  5.25it/s]


In [60]:
test_acc

52.12